In [1]:
import re
import nltk
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('stopwords')
import numpy as np
import pandas as pd
import heapq
import numpy as np
from collections import Counter

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Dataset Link:
* https://www.kaggle.com/listennotes/all-podcast-episodes-published-in-december-2017?select=podcasts.csv

In [2]:
podcasts_df = pd.read_csv('/content/podcasts.csv')

In [3]:
podcasts_df.head()

,uuid,title,image,description,language,categories,website,author,itunes_id
0,8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",http://is4.mzstatic.com/image/thumb/Music6/v4/...,Listen in as the Practical Ecommerce editorial...,English,Technology,http://www.practicalecommerce.com,Practical Ecommerce,874457373
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1015556393
2,73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,http://is5.mzstatic.com/image/thumb/Music71/v4...,A podcast about soundtracks and movies from my...,English,Podcasting | Technology,https://soundtrackalley.podbean.com,Randy Andrews,1158188937
3,0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,http://is1.mzstatic.com/image/thumb/Music71/v4...,The Tech M&A Podcast pulls from the best of th...,English,Business News | Technology | Tech News | Business,http://www.corumgroup.com,Timothy Goddard,538160025
4,69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",http://is4.mzstatic.com/image/thumb/Music62/v4...,The tech news show with two guys shooting the ...,English,Gadgets | Tech News | Technology,http://techinformist.com,The Tech Informist,916080498


In [4]:
podcasts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121175 entries, 0 to 121174
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uuid         121175 non-null  object
 1   title        121173 non-null  object
 2   image        121175 non-null  object
 3   description  119832 non-null  object
 4   language     121175 non-null  object
 5   categories   121175 non-null  object
 6   website      120005 non-null  object
 7   author       118678 non-null  object
 8   itunes_id    121175 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 8.3+ MB


In [5]:

podcasts_df.dropna(inplace=True)

In [6]:
podcasts_df=podcasts_df.iloc[0:9000].reset_index()


In [7]:
podcasts_df['text'] = podcasts_df[['title', 'author', 'categories', 'description']].apply(lambda x: " ".join(x), axis=1)
podcasts_df=podcasts_df[['title','text']]
podcasts_df.head()

,title,text
0,"Ecommerce Conversations, by Practical Ecommerce","Ecommerce Conversations, by Practical Ecommerc..."
1,Eat Sleep Code Podcast,Eat Sleep Code Podcast Telerik Tech News | Tec...
2,SoundtrackAlley,SoundtrackAlley Randy Andrews Podcasting | Tec...
3,The Tech M&A Podcast,The Tech M&A Podcast Timothy Goddard Business ...
4,"The Tech Informist - For fans of Apple, Google...","The Tech Informist - For fans of Apple, Google..."


In [8]:
podcasts_df['text'][0]

'Ecommerce Conversations, by Practical Ecommerce Practical Ecommerce Technology Listen in as the Practical Ecommerce editorial staff interviews interesting personalities in the ecommerce space.'

In [9]:
# create list of stop words
stop = stopwords.words('english')

# remove non-alphanumeric, non-space
stop = [re.sub(r'([^\s\w]|_)+', '', x) for x in stop]

# add in custom stop words
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']

months = ['january', 'february', 'march', 'april', 'may', 'june', 
          'july', 'august', 'september', 'october', 'november', 'december']

other = ['nan', 'podcast', 'podcasts', 'every', 'new', 'weekly', 
         'stories', 'story', 'episode', 'episodes', 'listen', 
         'host', 'hosted', 'join']

[stop.append(str(day)) for day in days]
[stop.append(str(month)) for month in months]
[stop.append(str(x)) for x in other]

def topKFrequent(tokenized_text, k): 
   
    count = Counter(tokenized_text)   
    
    return heapq.nlargest(k, count.keys(), key=count.get)

def remove_stop(text, stop):
    custom_stop = stop
#     top5 = topKFrequent(text, 5)
#     custom_stop = custom_stop + top5
    
    new_text = []
    for word in text:
        if word not in custom_stop:
            new_text.append(word)
    return new_text

# create tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# create stemmer
p_stemmer = PorterStemmer()
l_stemmer = WordNetLemmatizer() 


def stem_list(text, p_stemmer):
    new_list = []
    for word in text:
        new_list.append(p_stemmer.stem(word))
    return new_list

def lem_list(text, l_stemmer):
    new_list = []
    for word in text:
        new_list.append(l_stemmer.lemmatize(word))
    return new_list

def preprocess_text(text):
    # remove mixed alphanumeric
    text = re.sub(r"""(?x) # verbose regex
                            \b    # Start of word
                            (?=   # Look ahead to ensure that this word contains...
                             \w*  # (after any number of alphanumeric characters)
                             \d   # ...at least one digit.
                            )     # End of lookahead
                            \w+   # Match the alphanumeric word
                            \s*   # Match any following whitespace""", 
                             "", text)
    
    # remove urls (will check and remove http and www later)
    text = re.sub(r'\s([\S]*.com[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.org[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.net[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.edu[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.gov[\S]*)\b', '', text)
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    # tokenize text
    text = tokenizer.tokenize(text.lower())
    
    # remove stop words
    text = remove_stop(text, stop)
    
    # stem
    text = lem_list(text, l_stemmer)
    
    # remove instances of http or www
    new_text_list = []
    for word in text:
        if re.search(r'http', word):
            continue
        if re.search(r'www', word):
            continue
        new_text_list.append(word)
    
    new_text = ' '.join(new_text_list)
    
    return new_text

In [10]:
podcasts_df['text'] = podcasts_df['text'].map(preprocess_text)
podcasts_df = podcasts_df[podcasts_df.text != '']

In [11]:
podcasts_df

,title,text
0,"Ecommerce Conversations, by Practical Ecommerce",ecommerce conversation practical practical tec...
1,Eat Sleep Code Podcast,eat sleep code telerik tech news technology sh...
2,SoundtrackAlley,soundtrackalley randy andrew podcasting techno...
3,The Tech M&A Podcast,tech timothy goddard business news technology ...
4,"The Tech Informist - For fans of Apple, Google...",tech informist fan apple google microsoft tech...
...,...,...
8995,HHWLOD Master Feed,hhwlod master feed hhwlod network literature t...
8996,Horror Movie Night,horror movie night horror movie night comedy t...
8997,Hey Let's Talk About That Movie or Whatever,hey let talk movie whatever hey let talk movie...
8998,Let's Get Animated!,let get animated let get animated tv film kick...


In [12]:
def get_recommendations(title, sim_matrix):
    index=podcasts_df[podcasts_df['title']==title].index[0]
    distances=sorted(list(enumerate(sim_matrix[index])),reverse=True,key=lambda x:x[1])
    for i in distances[1:6]:
      print(podcasts_df.iloc[i[0]].title)                 


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

cv = CountVectorizer(max_features=5000,stop_words='english')
cv_matrix = cv.fit_transform(podcasts_df["text"]).toarray()
cv_cosine_sim = cosine_similarity(cv_matrix)

In [14]:
get_recommendations('Horror Movie Night', cv_cosine_sim)


The Terrible Terror Podcast
Movies Unhacked
Horrible Horror Podcast
Movie Improvie
Something Cheeky: Movies


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf_matrix = tf.fit_transform(podcasts_df["text"])
tf_cosine_sim = cosine_similarity(tf_matrix)

In [16]:
get_recommendations('Horror Movie Night', tf_cosine_sim)


United Nations of Horror
Horrible Horror Podcast
Music City Horror Podcast
Horrified Chicken
Nightmare On Film Street - A Horror Movie Podcast


In [17]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from matplotlib import pyplot

In [18]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [19]:

text_list = list(podcasts_df.text)
tokenized_text = [tokenizer.tokenize(i) for i in text_list]

In [20]:
w2v_model = Word2Vec(tokenized_text, sg=1)


In [21]:
nltk.download('punkt')
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v_model)
mean_embedded = mean_embedding_vectorizer.fit_transform(podcasts_df['text'])
w2v_cosine_sim = cosine_similarity(mean_embedded)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [22]:
get_recommendations('Horror Movie Night', w2v_cosine_sim)


Horrible Horror Podcast
Married With Movies
Have You Seen This One? (HYSTO?)
Rated M for MacPhail - 1001 Movies and Beyond
Decades Podcast


In [23]:
from scipy.sparse.linalg import svds
from sklearn.decomposition import TruncatedSVD

In [24]:
def remove_first_principal_component(X):
    svd = TruncatedSVD(n_components=1, n_iter=7, random_state=0)
    svd.fit(X)
    pc = svd.components_
    XX = X - X.dot(pc.transpose()) * pc
    return XX

def smooth_inverse_frequency(sent, a=0.001, word2vec_model=w2v_model):
    word_counter = {}
    sentences = []
    total_count = 0
    no_of_sentences = 0
    
    for s in sent:
        for w in s:
            if w in word_counter:
                word_counter[w] = word_counter[w] + 1
            else:
                word_counter[w] = 1
        total_count = total_count + len(s)
        no_of_sentences = no_of_sentences + 1
    
    sents_emd = []
    for s in sent:
        sent_emd = []
        for word in s:
            if word in word2vec_model:
                emd = (a/(a + (word_counter[word]/total_count)))*word2vec_model[word]
                sent_emd.append(emd)
        sum_ = np.array(sent_emd).sum(axis=0)
        sentence_emd = sum_/float(no_of_sentences)
        sents_emd.append(sentence_emd)
        
    new_sents_emb = remove_first_principal_component(np.array(sents_emd))
    return new_sents_emb

In [25]:
sif_text_emd = smooth_inverse_frequency(text_list)
sif_cosine_sim = cosine_similarity(sif_text_emd)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [26]:
get_recommendations('Horror Movie Night', sif_cosine_sim)


TransMissions - Everything Transformers!
Gold Ave Church Sermons
Trinity Evangelical Church
Freethought Radio
THE WEEKLY MOTOR
